In [2]:
import win32com.client
import subprocess
import time

In [ ]:
with open("credentials.txt", "r") as file:
    lines = file.readlines()
    # Read the first line for the client ID and remove extra whitespace
    client_id = lines[0].strip()
    # Read the Second line for the username and remove extra whitespace
    username = lines[1].strip()
    # Read the Third line line for the password and remove extra whitespace
    password = lines[2].strip()
    # Read the Fourth line for the Name and remove extra whitespace
    name = lines[3].strip()

# print(f"Username: {username}")
# print(f"Password: {password}")


In [ ]:
# open SAP Logon
subprocess.Popen(r"C:\\Program Files (x86)\SAP\\FrontEnd\SAPgui\saplogon.exe")
time.sleep(5)
SapGuiAuto = win32com.client.GetObject('SAPGUI')
application = SapGuiAuto.GetScriptingEngine
connection = application.OpenConnection(name,True)

# login
session = connection.Children(0)
session.findbyId("wnd[0]").maximize
session.findById("wnd[0]/usr/txtRSYST-MANDT").text = client_id # Client
session.findById("wnd[0]/usr/txtRSYST-BNAME").text = username    # User
session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = password  # Password
session.findById("wnd[0]/usr/txtRSYST-LANGU").text = "EN"   # Language
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [11]:
# navigate to transaction code
session.findById("wnd[0]/tbar[0]/okcd").text = "/nMB52" # Transaction code
session.findById("wnd[0]").sendVKey(0) # Enter
time.sleep(1)

In [8]:
# Entering Plant Number
session.findById("wnd[0]/usr/ctxtWERKS-LOW").text = "9100"
time.sleep(1)
session.findById("wnd[0]/usr/ctxtP_VARI").text = "/US9100-INV."
time.sleep(1)
session.findById("wnd[0]").sendVKey(8) # F8
time.sleep(1)


com_error: (-2147352567, 'Exception occurred.', (617, 'SAP Frontend Server', 'The virtual key is not enabled.', None, 0, 0), None)

In [ ]:
#### This function presses on the menu bar and triggers the export to spreadsheet option

# --- make sure the ALV grid has focus (menu items are enabled only then)
for pid in [
    "wnd[0]/usr/cntlGRID1/shellcont/shell",
    "wnd[0]/usr/cntlALV_GRID/shellcont/shell",
    "wnd[0]/usr/cntlRESULT_LIST/shellcont/shell",
]:
    try:
        session.findById(pid).setFocus()
        break
    except:
        pass

# --- helper that clicks a menu path by visible captions
def click_menu(*labels):
    path = "wnd[0]/mbar"
    for depth, want in enumerate(labels):
        want = want.lower()
        i = 0
        while True:
            try:
                itm = session.findById(f"{path}/menu[{i}]")
            except:
                raise Exception(f"Menu '{labels[depth]}' not found at depth {depth}.")
            # remove accelerators (&) and match by prefix
            txt = itm.text.replace("&", "").strip().lower()
            if txt.startswith(want):
                path = f"{path}/menu[{i}]"
                break
            i += 1
    # final select
    session.findById(path).select()

# --- List → Export → Spreadsheet…
click_menu("List", "Export", "Spreadsheet")



In [9]:
# --- Export dialog (OK) and Save As
try:
    session.findById("wnd[1]/tbar[0]/btn[0]").press()  # OK on 'Export' popup (if shown)
except: 
    pass

try:
    session.findById("wnd[1]/usr/ctxtDY_PATH").text = r"C:\Temp"
    session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "MB52_Export.xlsx"
    session.findById("wnd[1]/tbar[0]/btn[0]").press()  # Save
except:
    try:
        session.findById("wnd[2]/tbar[0]/btn[0]").press()  # Overwrite = Yes
    except:
        pass

In [10]:
# Adding Time Stamp as file name
from datetime import datetime
# Function to create time Stamp and add as Text
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
FileName = "DailyStockCount"
concat = FileName+"_"+timestamp

session.findById("wnd[1]/usr/ssubSUB_CONFIGURATION:SAPLSALV_GUI_CUL_EXPORT_AS:0512/txtGS_EXPORT-FILE_NAME").text = concat

# Clicking Export Button
session.findById("wnd[0]").sendVKey(0) # Enter



In [ ]:
session.findById("wnd[1]/usr/ctxtDY_PATH").text =r"C:\\Users\\JeSai\\OneDrive - Boviet Solar USA\Boviet IT Team - Boviet IT Team Channel\BI Dashboards\Stock Data"

session.findById("wnd[0]").sendVKey(0) # Enter
